In [1]:
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
import talib
!pip install mpl_finance
from google.colab import drive
drive.mount('/content/drive')

Selecting previously unselected package libta-lib0.
(Reading database ... 144611 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 276kB 2.7MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.19-cp36-cp36m-linux_x86_64.whl size=1437807 sha256=972fa9d80fc8aa043204145b20d2255aa9072fd942b500bdb8315a71786987b9
  Stored in directory: /root/.cache/pip/wheels/a3/f6/12/3d1ccd06caadd8fa47e016991dd0d27f1163bb260f1854e2ff
Successfully built ta-lib
Mounted at /

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import talib
import pandas_datareader as pdr
import mpl_finance as mpf
import seaborn as sns
import datetime as datetime
from sklearn import preprocessing
import requests
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from PIL import Image, ImageDraw, ImageFont
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
import csv
import statistics
import math
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SpatialDropout2D, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D,MaxPooling1D
from tensorflow.keras.layers import Activation,Dense ,Dropout ,GRU, ConvLSTM2D ,LSTM ,Bidirectional,TimeDistributed,Flatten, Conv1D, MaxPooling1D,BatchNormalization
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers 
from tensorflow.keras.layers import LayerNormalization
from tensorflow.python.keras.layers import Input, Embedding, Dot, Reshape, Dense
from tensorflow.python.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adamax, Adam
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

In [4]:
def exponential_smoothing(days, alpha, numdata):
  # assign weights
  weights = []
  i = days 
  while i>0:
    weights.append( math.exp(-alpha*i) )
    i-=1
  weights = np.array(weights) / sum(weights)
  # return data after smoothing
  results = []
  results += numdata[:days]
  for i in range(days, len(numdata)):
    results.append((np.dot( np.array(weights), np.array(numdata[i-days:i]))))
  return results

classes =['0', '1']
def to_onelist(text):
    onehot = [0] * 2
    onehot[ int(classes.index(str(text)) ) ] = 1
    label_list = np.array(onehot)
    return label_list

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)
start = '2005/01/01'
end = '2020/09/21'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

In [7]:
#選定欲回測的model
from keras.models import load_model
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)
model = load_model('/content/drive/My Drive/Program_Trading_Code/lstm4_5_5.h5')

stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002",
            "2105", "2207", #"2227", 
            "2301", "2303", "2308", "2317", "2327",
            "2330", "2352", "2357", "2382", "2395", "2408", "2412", "2454", "2474", "2609", "2610",
            "2633", "2801", "2823", "2880", "2881", "2882", "2883", "2884", "2885",
            "2886", "2887", "2888", "2890", "2891", "2892", "2912", "3008", "3045", "3711",
            "4904", "4938", "5871", "5880", "6505", "9904", "9910"]
# stockNum = ["2330"]

#設定 股票資料
#start = '2019/05/13'
start = '2018/08/01'
end = '2019/06/03'
#end = '2020/09/21'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

In [14]:
# validation 的報酬
DIC_ROR = {'1101': -8.6096, '1102': 10.1097, '1216': -0.7935, '1301': -9.4354, '1326': -17.9471, '1402': -11.2923, '2002': -9.5477, '2105': -22.0487, '2207': 75.6757, '2301': 13.9152, '2303': -1.3974, '2308': -4.2712, '2317': 12.161, '2327': -8.3967, '2330': -7.3642, '2352': -0.7237, '2357': -24.599, '2382': 13.4822, '2395': 4.3062, '2408': -15.2948, '2412': 0.2369, '2454': -12.0878, '2474': -10.7817, '2609': -5.6983, '2610': 16.8774, '2633': 44.6408, '2801': -4.7183, '2823': -16.7678, '2880': 5.6272, '2881': -15.3308, '2882': -20.0186, '2883': -11.0375, '2884': 21.6132, '2885': -4.1252, '2886': 18.3774, '2887': -6.0796, '2888': -17.4904, '2890': 6.5775, '2891': -14.73, '2892': 2.4412, '2912': -16.2974, '3008': 0.411, '3045': 2.3242, '3711': -20.7536, '4904': 2.2686, '4938': -0.4619, '5871': -1.0963, '5880': 1.4831, '6505': -15.977, '9904': 4.9289, '9910': -14.1742}

In [19]:
Weights_Dict = {}
for key in DIC_ROR:
  #print(key, '->', DIC_ROR[key])
  #篩選 validatiion 報酬率較高(n %)
  if DIC_ROR[key] > 5:
    Weights_Dict[key] = DIC_ROR[key]*DIC_ROR[key]
WeightSum = sum(Weights_Dict.values())
NewWeightSum = 0
for key in Weights_Dict:
  Weights_Dict[key] /= WeightSum
  print(key, '->', Weights_Dict[key])
  NewWeightSum += Weights_Dict[key]
print(Weights_Dict)
print(NewWeightSum)

1102 -> 0.010747504299808655
2207 -> 0.6022044835810461
2301 -> 0.020361510676109543
2317 -> 0.015551406294135584
2382 -> 0.01911404576751934
2610 -> 0.029953127681124257
2633 -> 0.2095535529939619
2880 -> 0.003329781935240672
2884 -> 0.049121230270953345
2886 -> 0.03551397031574665
2890 -> 0.004549386184354133
{'1102': 0.010747504299808655, '2207': 0.6022044835810461, '2301': 0.020361510676109543, '2317': 0.015551406294135584, '2382': 0.01911404576751934, '2610': 0.029953127681124257, '2633': 0.2095535529939619, '2880': 0.003329781935240672, '2884': 0.049121230270953345, '2886': 0.03551397031574665, '2890': 0.004549386184354133}
1.0


In [20]:
#決定好權重後， 開始test

#設定 股票資料
start = '2019/05/13'
# start = '2018/08/01'
# end = '2019/06/03'
end = '2020/09/21'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

In [21]:
asset_at_the_start = 19 * 10**6
total_asset_present_value = 0
for stock_num in stockNum:
  if stock_num not in Weights_Dict.keys(): continue
  #先算出之後回測用的價位
  Price_df = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=start, end=end)
  Price_df.index = Price_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
  #融資表格
  stock_time = str(Price_df.index[0]).split(" ")[0]
  # print(stock_time)
  #load in 融資融券
  url = "https://api.finmindtrade.com/api/v3/data"
  parameter = {
    "dataset": "TaiwanStockMarginPurchaseShortSale",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  data['b_total'] = data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']
  data['s_total'] = data['ShortSaleLimit'] - data['ShortSaleTodayBalance']
  data['b_s_ratio'] = (data['ShortSaleLimit'] - data['ShortSaleTodayBalance'])/(data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']+1)
  data = data.set_index('date')
  # 合併這兩個表格
  new_df = Price_df.join(data)
  new_df = new_df.drop(['stock_id'], axis=1)
  new_df = new_df.drop(['ShortSaleCashRepayment'], axis=1)
  # 處理三大法人投資情況 視情況加入
  url = "https://api.finmindtrade.com/api/v3/data"
  parameter = {
    "dataset": "InstitutionalInvestorsBuySell",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  df = pd.DataFrame()
  z = 0 
  i = 0
  while i < len(data)  :
    a = data['date'][i]
    count = 0
    sell = 0
    buy = 0 
    if i+6 < len(data)-1:
        for j  in range(i, i+6):
            if data['date'][j]== a :
              count += 1
              sell += data['sell'][j]
              buy += data['buy'][j]
    else:
      for j in range(i, len(data)):
        if data['date'][j] == a :
          count += 1
          sell += data['sell'][j]
          buy += data['buy'][j]
    df = df.append({'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}, ignore_index=True)
    i+=count
  new_df = new_df.join(df.set_index('date'))
  new_df = new_df.fillna(axis=0,method='ffill')
  new_df = new_df.drop(['Note'], axis=1)
  # gpy = data.groupby('date').sum()
  # gpy.rename(columns={'buy':'buy_volume', 'sell': 'sell_volume'}, inplace=True)
  # gpy['bs_ratio'] = gpy['buy_volume'] / (gpy['sell_volume']+1)
  # df = gpy
  # new_df = new_df.join(df)
  # new_df = new_df.dropna(axis=0, how='any')
  # new_df = new_df.drop(['Note'], axis=1)
  new_df.reset_index(drop=True, inplace=True)
  for columnName in new_df.columns:
    if str(columnName) == 'y_label' or str(columnName) == 'index': continue
    # print(columnName)
    new_df[str(columnName)] = exponential_smoothing(5, 0.2, list(new_df[str(columnName)]))
    days = 10
    ans = [0]*days
    for i in range(days,len(new_df)):
      Pi = new_df['Close'][i]
      Pj = new_df['Close'][i-days]
      if Pi > Pj : y = 1
      else: y = 0
      ans.append(y)
  new_df['y_label']= ans
  #更改index 將日期更改為
  new_df = new_df.reset_index()
  new_df.rename(columns={'index':'Date'}, inplace=True)
  time_list = list(new_df['Date'])
  new_df = new_df.drop(['Date'], axis=1)
  new_df = new_df.drop(['MarginPurchaseLimit'], axis=1)
  new_df = new_df.drop(['ShortSaleLimit'], axis=1)
  # new_df = new_df.drop(['level_0'], axis=1)
  Price_df = Price_df.reset_index()

  # start evaluaion 
  ###############一般回測################################
  if stock_num not in Weights_Dict.keys(): continue
  left_money = asset_at_the_start * Weights_Dict[stock_num] #10**6
  own_asset = left_money #10**6
  time_slide = 30 
  own_stock = 0 
  money_record = [left_money]*time_slide
  #origin_money = 10**6 #紀錄開場進最後出
  origin_hold_stock = 0 
  for i in range(time_slide, len(new_df)): 
    tmp = new_df[i-time_slide:i]
    for col in new_df.columns:
      if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
      mean = statistics.mean(tmp[str(col)])
      stdev = statistics.stdev(tmp[str(col)])
      if stdev == 0: stdev = 1
      def test(s): 
        return (float(s)-mean)/stdev
      tmp[str(col)].map(test)
    x = np.array(np.array(tmp).tolist()[0: time_slide]).reshape(time_slide,len(tmp.columns))
    predict = int(classes[np.argmax(model.predict(x.reshape(1,30,23)))])
    price = int(Price_df['Close'][i]*100)/100
    # if i == time_slide: #起始買
    #   origin_hold_stock += int( left_money / ( price * (1 + 0.001425)) ) # 包含手續費可以購買的股數
    #   origin_money = origin_money - origin_hold_stock*price*(1 + 0.001425)
    # elif i == len(new_df)-1: #結束賣
    #   origin_money = origin_money + origin_hold_stock*price*(1 - 0.001425 - 0.003)
    if predict == 1:
      if own_stock ==0:
        # print("買入")
        own_stock += int( left_money / ( price * (1 + 0.001425)) )
        left_money = left_money - own_stock*price*(1 + 0.001425)
        # print("目前價格： " , price, "日期: ",time_list[i])
        # print("現有資產： ", (left_money+own_stock*price),"擁有股票：", own_stock, "資金部分： ", left_money)
    elif predict == 0 :
      if own_stock > 0:
        # print("賣出")
        left_money += own_stock*price*(1 - 0.001425 - 0.003)
        own_stock = 0
        # print("目前價格： " , price, "日期: ", time_list[i])
        # print("現有資產： ", (left_money+own_stock*price),"擁有股票：", own_stock, "資金部分： ", left_money)
    money_record.append(left_money+own_stock*price)
  print(str(stock_num)+"最後資產 ：", int(money_record[-1]),"報酬率：",(int(money_record[-1])-(own_asset))*100/(own_asset),"%")
  total_asset_present_value += int(money_record[-1])
print("測試結束")
print("our總投資報酬率：", (total_asset_present_value - asset_at_the_start)*100 / asset_at_the_start, "%")

1102最後資產 ： 156903 報酬率： -23.16306743207378 %
2207最後資產 ： 22489655 報酬率： 96.55550313953756 %
2301最後資產 ： 348311 報酬率： -9.966612073404596 %
2317最後資產 ： 297433 報酬率： 0.6620759883038573 %
2382最後資產 ： 368381 報酬率： 1.4357395604729895 %
2610最後資產 ： 538070 報酬率： -5.454034764934482 %
2633最後資產 ： 4354026 報酬率： 9.355942614104844 %
2880最後資產 ： 62689 報酬率： -0.9117979254968435 %
2884最後資產 ： 1067029 報酬率： 14.328205427379322 %
2886最後資產 ： 718627 報酬率： 6.500268339302806 %
2890最後資產 ： 71232 報酬率： -17.59212166967988 %
測試結束
our總投資報酬率： 60.38082105263158 %
